In [1]:
# Deribit Real-time Data Updater for Google Sheets - Jupyter Notebook Version
# Run this in Jupyter Notebook for interactive monitoring

# Cell 1: Install required packages (run once)
# Uncomment and run this cell if packages are not installed
"""
!pip install requests gspread google-auth google-auth-oauthlib google-auth-httplib2
!pip install ipywidgets  # For interactive widgets
"""

# Cell 2: Import libraries
import requests
import time
import gspread
from google.oauth2.service_account import Credentials
import json
from datetime import datetime
import logging
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
from threading import Thread
import asyncio

# Cell 3: Setup logging for Jupyter
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Cell 4: Configuration
# UPDATE THESE VALUES FOR YOUR SETUP
CONFIG = {
    "CREDENTIALS_FILE": "/var/www/gat/jupyter_notebook/service-account-key.json",
    "SHEET_NAME": "Deribit IR calculator",
    "WORKSHEET_NAME": "Sheet1",       
    "INSTRUMENT": "BTC-PERPETUAL",
    "UPDATE_INTERVAL": 10,
}

print("Configuration loaded:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

# Cell 5: Main DeribitUpdater class
class DeribitJupyterUpdater:
    def __init__(self, config):
        self.config = config
        self.deribit_base_url = "https://www.deribit.com/api/v2/public"
        self.running = False
        self.latest_data = {}
        self.data_history = []
        self.setup_google_sheets()
        
    def setup_google_sheets(self):
        """Setup Google Sheets API connection"""
        try:
            scope = [
                "https://spreadsheets.google.com/feeds",
                "https://www.googleapis.com/auth/drive"
            ]
            
            creds = Credentials.from_service_account_file(
                self.config["CREDENTIALS_FILE"], scopes=scope
            )
            
            self.gc = gspread.authorize(creds)
            self.sheet = self.gc.open(self.config["SHEET_NAME"])
            self.worksheet = self.sheet.worksheet(self.config["WORKSHEET_NAME"])
            
            print(f"✅ Successfully connected to Google Sheet: {self.config['SHEET_NAME']}")
            
        except Exception as e:
            print(f"❌ Failed to setup Google Sheets: {e}")
            raise
    
    def fetch_deribit_data(self, instrument=None):
        """Fetch real-time data from Deribit API"""
        if instrument is None:
            instrument = self.config["INSTRUMENT"]
            
        try:
            # Get ticker data
            ticker_url = f"{self.deribit_base_url}/ticker"
            ticker_params = {"instrument_name": instrument}
            ticker_response = requests.get(ticker_url, params=ticker_params, timeout=5)
            ticker_data = ticker_response.json()
            
            if ticker_response.status_code != 200:
                print(f"❌ Ticker API error: {ticker_data}")
                return None
            
            # Get index price
            index_name = instrument.split('-')[0].lower() + "_usd"
            index_url = f"{self.deribit_base_url}/get_index_price"
            index_params = {"index_name": index_name}
            index_response = requests.get(index_url, params=index_params, timeout=5)
            index_data = index_response.json()
            
            if index_response.status_code != 200:
                print(f"❌ Index API error: {index_data}")
                return None
            
            # Calculate premium rate
            mark_price = ticker_data["result"]["mark_price"]
            index_price = index_data["result"]["index_price"]
            premium_rate = ((mark_price - index_price) / index_price) * 100
            
            # Prepare result
            result = {
                "timestamp": datetime.now(),
                "mark_price": mark_price,
                "index_price": index_price,
                "funding_8h": ticker_data["result"]["funding_8h"],
                "premium_rate": premium_rate,
                "best_bid": ticker_data["result"]["best_bid_price"],
                "best_ask": ticker_data["result"]["best_ask_price"],
                "last_price": ticker_data["result"]["last_price"],
                "open_interest": ticker_data["result"]["open_interest"],
                "volume_24h": ticker_data["result"]["stats"]["volume"],
                "instrument": instrument
            }
            
            return result
            
        except Exception as e:
            print(f"❌ Error fetching data: {e}")
            return None
    
    def update_google_sheet(self, data):
        """Update Google Sheet with fetched data"""
        try:
            if not data:
                return False
            
            updates = [
                ("B1", data["mark_price"]),
                ("B2", data["index_price"]),
                ("B7", data["funding_8h"]),
                ("B8", data["timestamp"].strftime("%Y-%m-%d %H:%M:%S")),
                ("B9", data["best_bid"]),
                ("B10", data["best_ask"]),
                ("B11", data["premium_rate"]),
            ]
            
            cells_to_update = []
            for cell, value in updates:
                cells_to_update.append({
                    'range': cell,
                    'values': [[value]]
                })
            
            self.worksheet.batch_update(cells_to_update)
            return True
            
        except Exception as e:
            print(f"❌ Failed to update Google Sheet: {e}")
            return False
    
    def single_update(self):
        """Perform a single data fetch and update"""
        data = self.fetch_deribit_data()
        if data:
            self.latest_data = data
            self.data_history.append(data)
            
            # Keep only last 100 records
            if len(self.data_history) > 100:
                self.data_history = self.data_history[-100:]
            
            success = self.update_google_sheet(data)
            
            return data, success
        return None, False
    
    def start_continuous_updates(self):
        """Start continuous updates in background"""
        self.running = True
        
        def update_loop():
            while self.running:
                try:
                    data, success = self.single_update()
                    if data:
                        status = "✅ Updated" if success else "⚠️ Fetch OK, Sheet Failed"
                        print(f"{data['timestamp'].strftime('%H:%M:%S')} | "
                              f"Mark: {data['mark_price']:,.2f} | "
                              f"Index: {data['index_price']:,.2f} | "
                              f"Premium: {data['premium_rate']:.4f}% | "
                              f"Funding: {data['funding_8h']:.4f}% | {status}")
                    else:
                        print(f"{datetime.now().strftime('%H:%M:%S')} | ❌ Failed to fetch data")
                    
                    time.sleep(self.config["UPDATE_INTERVAL"])
                    
                except Exception as e:
                    print(f"❌ Error in update loop: {e}")
                    time.sleep(5)
        
        thread = Thread(target=update_loop, daemon=True)
        thread.start()
        print(f"🚀 Started continuous updates every {self.config['UPDATE_INTERVAL']} seconds")
        print("📊 Real-time data will appear below...")
        return thread
    
    def stop_updates(self):
        """Stop continuous updates"""
        self.running = False
        print("⏹️ Stopping continuous updates...")
    
    def get_data_summary(self):
        """Get summary of recent data"""
        if not self.data_history:
            return "No data available"
        
        df = pd.DataFrame(self.data_history)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        return df.tail(10)[['timestamp', 'mark_price', 'index_price', 'premium_rate', 'funding_8h']]

# Cell 6: Initialize the updater
try:
    updater = DeribitJupyterUpdater(CONFIG)
    print("🎉 Deribit Updater initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize: {e}")
    updater = None

# Cell 7: Control Panel with Buttons
if updater:
    # Create control buttons
    start_button = widgets.Button(description="🚀 Start Updates", button_style="success")
    stop_button = widgets.Button(description="⏹️ Stop Updates", button_style="danger")
    single_button = widgets.Button(description="📊 Single Update", button_style="info")
    summary_button = widgets.Button(description="📈 Show Summary", button_style="warning")
    
    output = widgets.Output()
    
    def on_start_click(b):
        with output:
            clear_output()
            if not updater.running:
                updater.start_continuous_updates()
            else:
                print("Updates already running!")
    
    def on_stop_click(b):
        with output:
            updater.stop_updates()
    
    def on_single_click(b):
        with output:
            print("Fetching single update...")
            data, success = updater.single_update()
            if data:
                print(f"✅ Success: Mark={data['mark_price']}, Index={data['index_price']}, "
                      f"Premium={data['premium_rate']:.4f}%, Funding={data['funding_8h']:.4f}%")
            else:
                print("❌ Failed to fetch data")
    
    def on_summary_click(b):
        with output:
            clear_output()
            summary = updater.get_data_summary()
            print("📊 Recent Data Summary:")
            print(summary)
    
    start_button.on_click(on_start_click)
    stop_button.on_click(on_stop_click)
    single_button.on_click(on_single_click)
    summary_button.on_click(on_summary_click)
    
    # Display control panel
    control_panel = widgets.HBox([start_button, stop_button, single_button, summary_button])
    display(control_panel)
    display(output)
    
    print("🎛️ Use the buttons above to control the updater")

# Cell 8: Manual testing functions
def test_single_fetch():
    """Test a single data fetch"""
    if updater:
        data, success = updater.single_update()
        if data:
            print("✅ Test successful!")
            print(f"Mark Price: {data['mark_price']:,.2f}")
            print(f"Index Price: {data['index_price']:,.2f}")
            print(f"Premium Rate: {data['premium_rate']:.4f}%")
            print(f"Funding Rate: {data['funding_8h']:.4f}%")
            print(f"Sheet Updated: {'Yes' if success else 'No'}")
        else:
            print("❌ Test failed!")

def show_current_data():
    """Show the most recent data"""
    if updater and updater.latest_data:
        data = updater.latest_data
        print(f"🕒 Last Update: {data['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 Mark Price: {data['mark_price']:,.2f}")
        print(f"📈 Index Price: {data['index_price']:,.2f}")
        print(f"💰 Premium Rate: {data['premium_rate']:.4f}%")
        print(f"🏦 Funding Rate (8h): {data['funding_8h']:.4f}%")
    else:
        print("No data available yet. Run a single fetch first.")

# Cell 9: Example usage
print("""
🚀 Jupyter Notebook Deribit Updater Ready!

Quick Start:
1. Click '🚀 Start Updates' to begin continuous monitoring
2. Click '📊 Single Update' to test one update
3. Click '📈 Show Summary' to see recent data
4. Click '⏹️ Stop Updates' to stop monitoring

Manual functions:
- test_single_fetch() - Test API connection
- show_current_data() - Show latest data
""")

Configuration loaded:
  CREDENTIALS_FILE: /var/www/gat/jupyter_notebook/service-account-key.json
  SHEET_NAME: Deribit IR calculator
  WORKSHEET_NAME: Sheet1
  INSTRUMENT: BTC-PERPETUAL
  UPDATE_INTERVAL: 10
✅ Successfully connected to Google Sheet: Deribit IR calculator
🎉 Deribit Updater initialized successfully!


Output()

🎛️ Use the buttons above to control the updater

🚀 Jupyter Notebook Deribit Updater Ready!

Quick Start:
1. Click '🚀 Start Updates' to begin continuous monitoring
2. Click '📊 Single Update' to test one update
3. Click '📈 Show Summary' to see recent data
4. Click '⏹️ Stop Updates' to stop monitoring

Manual functions:
- test_single_fetch() - Test API connection
- show_current_data() - Show latest data

10:33:55 | Mark: 108,865.63 | Index: 108,826.27 | Premium: 0.0362% | Funding: 0.0001% | ✅ Updated
10:34:05 | Mark: 108,863.81 | Index: 108,824.27 | Premium: 0.0363% | Funding: 0.0001% | ✅ Updated
10:34:16 | Mark: 108,863.38 | Index: 108,823.13 | Premium: 0.0370% | Funding: 0.0001% | ✅ Updated
10:34:26 | Mark: 108,853.08 | Index: 108,809.95 | Premium: 0.0396% | Funding: 0.0001% | ✅ Updated
10:34:37 | Mark: 108,861.90 | Index: 108,817.04 | Premium: 0.0412% | Funding: 0.0001% | ✅ Updated
10:34:47 | Mark: 108,861.70 | Index: 108,817.70 | Premium: 0.0404% | Funding: 0.0001% | ✅ Updated
10:34:5